# Analytische Lösung Zweifeldträgers{#sec-zweifeldtraeger_anhang}

Die analytische Lösung des Zweifeldträgers zeigt die ausführlichen Berechnungen des Vorlesungsbeispiels.

## Parameter

Zunächst sind die Parameter der Betonstähle, der Geometrie und des Betons aufgelistet.

In [1]:
from packages import *

### Bewehrung

Der Durchmesser und die Teilung der Bewehrung sind die folgenden:

In [2]:
# Längsbewehrung in X-Richtung gerichtet
diam_x = 14 * un.mm

# Teilung der Längsbewehrung
s_x = 200 * un.mm

# Längsbewehrung in Y-Richtung gerichtet
diam_y = 14 * un.mm

# Teilung der Längsbewehrung
s_y = 200 * un.mm

put_out(symbolic=False, rows = 2)

$$
\begin{aligned}
\oslash_{x}& = 14 \ \mathrm{mm} \quad & s_{x}& = 200 \ \mathrm{mm} \\ 
\oslash_{y}& = 14 \ \mathrm{mm} \quad & s_{y}& = 200 \ \mathrm{mm} \end{aligned}
$$

Die Querschnittsfläche der Längsbewehrung für den Meterstreifen beträgt dabei:

In [3]:
# Querschnittsfläche der Längsbewehrung, pro Längeneinheit
a_s = (diam_x**2 / 4 * np.pi / s_x).to(un.mm**2 / un.m)

put_out(symbolic=True)

$$
\begin{aligned}
a_{s}& = \frac{\oslash_{x}^{2} \cdot \pi}{4 \cdot s_{x}} = 769.69 \ \frac{\mathrm{mm}^{2}}{\mathrm{m}} \quad &  \quad &  
 \end{aligned}
$$

### Geometrie

Die Parameter der Geometrie beinhaltet die Betondeckung, die Plattenstärke, die Querschnittsbreite und die Balkenlänge:

In [4]:
# Betonüberdeckung
c = 20 * un.mm

# Plattenstärke
h = 300 * un.mm

# Streifenbreite
b = 1 * un.m

# Balkenlänge
l = 8 * un.m


put_out(rows= 2)

$$
\begin{aligned}
c& = 20 \ \mathrm{mm} \quad & h& = 300 \ \mathrm{mm} \\ 
b& = 1 \ \mathrm{m} \quad & l& = 8 \ \mathrm{m} \end{aligned}
$$

Daraus lässt sich die statische Höhe der Längsbewehrung bestimmen:

In [5]:
# Statische Höhe in der Bewehrung in X-Richtung
d_x = h-c-diam_y - diam_x/2

put_out(symbolic=True, evaluate=False)

$$
\begin{aligned}
d_{x}& = h - c - \oslash_{y} - \frac{\oslash_{x}}{2} = 259.0 \ \mathrm{mm} \quad &  \quad &  
 \end{aligned}
$$

### Beton

Der Beton ist mit der Zylinderdruckfestigkeit, Elastizitätsmodul und Bruchstauchung beschrieben.

In [6]:
# Zylinderdruckfestigkeit vorausgesetzt
f_cc = 30 * un.N / un.mm**2

# Elastizitätsmodul des Betons
E_c = 29.3 * un.kN / un.mm**2

# Bruchstauchung Erfahrungswert
eps_cu = 5 * un.permille

put_out()


$$
\begin{aligned}
f_{cc}& = 30.0 \ \frac{\mathrm{N}}{\mathrm{mm}^{2}} \quad & E_{c}& = 29.3 \ \frac{\mathrm{kN}}{\mathrm{mm}^{2}} \quad & \varepsilon_{cu}& = 5 \ \mathrm{‰} \end{aligned}
$$

Mittels empirischen Ansätzen lassen sich die Betonzugfestigkeit und die Bauteildruckfestigkeit bestimmen.

In [7]:
# Betonzugfestigkeit, nicht einheitentreu
f_ct = (0.3 * f_cc.m ** (2 / 3)) * un.N / un.mm**2

# Effektive Betondruckfestigkeit, nicht einheitentreu
f_c = (2.5 * f_cc.m ** (2 / 3)) * un.N / un.mm**2


put_out(symbolic=True, evaluate=False, rows=1)

$$
\begin{aligned}
f_{ct}& = 0.3 \cdot f_{cc}^{\frac{2}{3}} = 2.9 \ \frac{\mathrm{N}}{\mathrm{mm}^{2}} \\ 
f_{c}& = 2.5 \cdot f_{cc}^{\frac{2}{3}} = 24.14 \ \frac{\mathrm{N}}{\mathrm{mm}^{2}} \end{aligned}
$$

### Zugversteifung

Anhand der Betonzugfestigkeit ist die Schubspannungs-Schlupf-Beziehung zu definieren. Die Schubspannungen sind die folgenden.

In [8]:
# Verbundschubspannung vor dem Fliessen
tau_b0 = 2 * f_ct

# Verbundschubspannung vor dem Fliessen
tau_b1 = f_ct

put_out(symbolic=True, rows=2)

$$
\begin{aligned}
\tau_{b0}& = 2 \cdot f_{ct} = 5.79 \ \frac{\mathrm{N}}{\mathrm{mm}^{2}} \quad & \tau_{b1}& = f_{ct} = 2.9 \ \frac{\mathrm{N}}{\mathrm{mm}^{2}} \end{aligned}
$$

In [9]:
fig_tau_delta, ax_tau_delta = plt.subplots()

delta_array = np.array([0,0.0001,5,5.0001,10, 10.0001])
tau_array = np.array([0,tau_b0.m, tau_b0.m, tau_b1.m, tau_b1.m, 0])
ax_tau_delta.plot(delta_array, tau_array, color='black')

# Add labels and show the plot
ax_tau_delta.set_ylabel("$\\tau$ [N/mm$^2$]")
ax_tau_delta.set_xlabel("$\delta$")
ax_tau_delta.set_xticks([])# Save the plot to the specified file
plt.savefig("imgs/jag_tau_delta.svg")
plt.close()

Das Zugglied weist dabei die folgende Höhe, der entsprechende Bewehrungsgehalt und die daraus resultierenden Grenzen des Rissabstands auf. 

In [10]:
# Abschätzung der mitwirkenden Höhe der Betonzugzone
h_eff = 2 * (h - d_x)

# Geometrischer Bewehrungsgehalt
rho = (a_s / h_eff).to(un.percent)

# maximaler Rissabstand
s_r0 = (diam_x * (1 - rho) / (4 * rho)).to(un.mm)

# minimaler Rissabstand
s_rl = s_r0 / 2

put_out(symbolic=True, rows=1, evaluate=False)

$$
\begin{aligned}
h_{eff}& = 2 \cdot \left(h - d_{x}\right) = 82.0 \ \mathrm{mm} \\ 
\rho& = \frac{a_{s}}{h_{eff}} = 0.94 \ \mathrm{\%} \\ 
s_{r0}& = \frac{\oslash_{x} \cdot \left(1 - \rho\right)}{4 \cdot \rho} = 369.38 \ \mathrm{mm} \\ 
s_{rl}& = \frac{s_{r0}}{2} = 184.69 \ \mathrm{mm} \end{aligned}
$$

Es ist naheliegend den Rissabstand entsprechend der Stabteilung zu wählen, da die Bewehrung eine Betonschwächung darstellt, sich somit die Risse bei den Schwachstellen bilden. Der verwendete Rissabstand ist schliesslich der folgende.

In [11]:
s_r = 200 * un.mm

put_out()

$$
\begin{aligned}
s_{r}& = 200 \ \mathrm{mm} \quad &  \quad &  
 \end{aligned}
$$

In [12]:
# Bestimmung der mittleren Stahldehnung

def eps_sm_bilinear(
    E_s, E_sh, s_r, tau_b0, tau_b1, diam, sigma_sr, f_sy, f_su, epsilon_sy
):
    # display(Markdown("Berechnungen nach @marti_tragverhalten_1999 Seite 156"))
    if sigma_sr <= f_sy:
        # display(Markdown("Gleichung 1"))
        return sigma_sr / E_s - tau_b0 * s_r / (E_s * diam)

    if f_sy <= sigma_sr <= (f_sy + (2 * tau_b1 * s_r) / diam):
        # display(Markdown("Gleichung 2"))
        return (
            (sigma_sr - f_sy) ** 2
            * diam
            / (4 * E_sh * tau_b1 * s_r)
            * (1 - E_sh * tau_b0 / (E_s * tau_b1))
            + (sigma_sr - f_sy) / E_s * tau_b0 / tau_b1
            + (epsilon_sy - tau_b0 * s_r / (E_s * diam))
        )

    if (f_sy + 2 * tau_b1 * s_r / diam) <= sigma_sr <= f_su:
        # display(Markdown("Gleichung 3"))
        return (sigma_sr - f_sy) / E_sh + (epsilon_sy - tau_b1 * s_r / (E_sh * diam))
    



## Kaltverformter Betonstahl - B500A

Die Berechnung des kaltverfomten Betonstahls startet mit den Eigenschaften des Betonstahls. Gefolgt von der Bestimmung der gerissenen Biegesteifigkeit mit zugehörigem Hebelarm der inneren Kräfte.

Danach folgen die Grenzzustände des Zugglieds und die Systemanalyse. Die Systemanalyse beinhaltet 3 Zustände. 

### Eigenschaften des Betonstahls

Die Eigenschaften für den Betonstahl B500A sind mit einem Index $A$ gekennzeichnet.


In [13]:
# Fliessgrenze
f_sy_com_A = 570 * un.N / un.mm**2

# Zugfestigkeit
f_su_com_A = 600 * un.N / un.mm**2

# Fliessdehnung
eps_sy_com_A = 2.78 * un.permille

# Bruchdehnung
eps_su_com_A = 25 * un.permille

# Elastizitätsmodul
E_s_com_A = 205 * un.kN / un.mm**2

# Verfestigungsmodul
E_sh_com_A = 1350 * un.N / un.mm**2

put_out()

$$
\begin{aligned}
f_{sy , A}& = 570.0 \ \frac{\mathrm{N}}{\mathrm{mm}^{2}} \quad & f_{su , A}& = 600.0 \ \frac{\mathrm{N}}{\mathrm{mm}^{2}} \quad & \varepsilon_{sy , A}& = 2.78 \ \mathrm{‰} \\ 
\varepsilon_{su , A}& = 25 \ \mathrm{‰} \quad & E_{s , A}& = 205.0 \ \frac{\mathrm{kN}}{\mathrm{mm}^{2}} \quad & E_{sh , A}& = 1350.0 \ \frac{\mathrm{N}}{\mathrm{mm}^{2}} \end{aligned}
$$

Zusammengefasst sind die Eigenschaften im Spannungs-Dehnungs-Diagramm in der @fig-jag_stress_strain_b500a.

In [14]:
# | echo: false

stress_b500a_array = np.array([0, f_sy_com_A.m, f_su_com_A.m]) * un.MPa
strain_b500a_array = np.array([0, eps_sy_com_A, eps_su_com_A])

fig_b500a, ax_b500a = plt.subplots()
ax_b500a.plot(strain_b500a_array, stress_b500a_array.to(un.MPa).m, color="k")

# Scientific Notation if needed
apply_scientific_notation(ax_b500a, stress_b500a_array.m)

# Set axis labels
ax_b500a.set_ylabel(r"$\sigma$ [N/mm$^2$]")
ax_b500a.set_xlabel(r"$\varepsilon_s$ [-]")

# Set limits
ax_b500a.set_xlim(xmax=135 / 1000)
ax_b500a.set_ylim(ymax=750)

# Show the plot
# plt.tight_layout()
plt.savefig("imgs/jag_stress_strain_b500a.svg")
plt.close()

### Gerissene Biegesteifigkeit

Anhand der Querschnittsanalyse des Zustands $II$, gemäss der @fig-jag_qs_steifigkeit lässt sich die gerissene Biegesteifigkeit ermitteln.

![Querschnittsanalyse für die gerissene Biegesteifigkeit, Beton völlig elastisch, Bewehrung maximal bei der Fliessspannung](../imgs/jag_qs_steifigkeit.svg){#fig-jag_qs_steifigkeit}


Anhand der folgenden Wertigkeit lässt sich der Querschnitt als homogener Betonquerschnitt betrachten.


In [15]:
# Wertigkeit der Elastizitätsmodule
n = E_s_com_A / E_c

put_out(symbolic=True)

$$
\begin{aligned}
n& = \frac{E_{s , A}}{E_{c}} = 7.0 \  \quad &  \quad &  
 \end{aligned}
$$

Mittels dem Gleichgewicht der horizontalen Kräfte folgt die Betondruckzone:

In [16]:
# Mittels Sympy (CAS für Python) wird eine analytische Lösung der
# Druckzonenhöhe für den gerissenen Querschnitt ermittelt.

# Summe der horizontalen Kräfte
sum_Fx = sp.sympify("1/2*x*b_w*sigma_c -sigma_s*a_s*b_w")

# Definition der Spannung in Abhängigkeit der Dehnungen
expr_sigma_c = sp.sympify("eps_c * E_c")
expr_sigma_s = sp.sympify("eps_s * n * E_c")

# Die Beziehung zwischen der Stahl- und Betondehnung nach Hook
expr_epsilon_c = sp.sympify("eps_s /(d-x)*x")

# Substitution der Gleichungen
sum_Fx_subs = sum_Fx.subs(
    {
        "sigma_c": expr_sigma_c.subs("eps_c", expr_epsilon_c),
        "sigma_s": expr_sigma_s,
    }
)

# Erstellen einer numerischen Funktion
x_solve = sp.solve(sum_Fx_subs, "x")[1]
x_solve_lambd = sp.lambdify(("a_s", "b_w", "n", "d"), x_solve)


In [17]:

x_II = (np.sqrt(a_s * n * (a_s*n + 2 * d_x))-a_s * n).to(un.mm)
put_out(symbolic=True)

$$
\begin{aligned}
x_{II}& = \sqrt{a_{s} \cdot n \cdot \left(a_{s} \cdot n + 2 \cdot d_{x}\right)} - a_{s} \cdot n = 47.7 \ \mathrm{mm} \quad &  \quad &  
 \end{aligned}
$$

Daraus folgt das Flächenträgheitsmoment:

In [18]:
# Flächenmoment 2. Grades für den Beton
I_cII = (b * x_II**3 / 12 + b * x_II * (x_II / 2) ** 2).to(un.cm**4)

# Flächenmoment 2. Grades für den Betonstahl
I_sII = (a_s * b * n * (d_x - x_II) ** 2).to(un.cm**4)

# Flächenmoment 2. Grades für den gerissenen Querschnitt
I_II = (I_cII + I_sII).to(un.cm**4)

put_out(symbolic=True, rows=1, evaluate=False, precision=1)

$$
\begin{aligned}
I_{cII}& = \frac{b \cdot x_{II}^{3}}{12} + b \cdot x_{II} \cdot \left(\frac{x_{II}}{2}\right)^{2} = 3618.8 \ \mathrm{cm}^{4} \\ 
I_{sII}& = a_{s} \cdot b \cdot n \cdot \left(d_{x} - x_{II}\right)^{2} = 24042.6 \ \mathrm{cm}^{4} \\ 
I_{II}& = I_{cII} + I_{sII} = 27661.4 \ \mathrm{cm}^{4} \end{aligned}
$$

Und die gerissene Biegesteifigkeit bestimmt sich zu:

In [19]:
# Gerissene Biegesteifigkeit
EI_II_com_A = (E_c * I_II).to(un.kN * un.m**2)

put_out(symbolic=True)

$$
\begin{aligned}
EI_{II , A}& = E_{c} \cdot I_{II} = 8104.79 \ \mathrm{kN} \cdot \mathrm{m}^{2} \quad &  \quad &  
 \end{aligned}
$$

### Hebelarm der inneren Kräfte

Der Hebelarm der inneren Kräfte gilt entlang des Trägers als konstant.
Dabei ist dem Beton die Zylinderdruckfestigkeit hinterlegt, sowie erreicht der Stahl die Zugfestigkeit. Dargestellt ist der Querschnitt in der @fig-jag_qs_hebelarm.

![Querschnittsanalyse zur Bestimmung des Hebelarms der inneren Kräfte, Stahl erreicht Zugfestigkeit, Beton ist völlig plastifiziert](../imgs/jag_qs_hebelarm.svg){#fig-jag_qs_hebelarm}

Daraus resultiert die Druckzonenhöhe, der Hebelarm und der Biegewiderstand.

In [20]:
# Druckzonenhöhe für den Biegewiderstand
x_u_com_A = (a_s * b * f_su_com_A / (0.85 * f_cc * b)).to(un.mm)

# Hebelarm der inneren Kräfte für den Biegewiderstand
z_com_A = d_x - 0.85 * x_u_com_A / 2 

# Biegewiderstand
m_u_com_A = (a_s * f_su_com_A * z_com_A).to(un.kNm / un.m)

put_out(symbolic=True, evaluate=False, rows=1)

$$
\begin{aligned}
x_{u , A}& = \frac{a_{s} \cdot b \cdot f_{su , A}}{0.85 \cdot f_{cc} \cdot b} = 18.11 \ \mathrm{mm} \\ 
z_{, A}& = d_{x} - \frac{0.85 \cdot x_{u , A}}{2} = 251.3 \ \mathrm{mm} \\ 
m_{u , A}& = a_{s} \cdot f_{su , A} \cdot z_{, A} = 116.06 \ \frac{\mathrm{kNm}}{\mathrm{m}} \end{aligned}
$$

### Grenzzustände des Zugglieds

Nun folgen zwei Grenzzustände des Zugglieds. Zum Einen bei Fliessbeginn, zum anderen beim Erreichen der Zugfestigkeit des Betonstahls. Dies dient zur Bestimmung des Rotationsvermögens.

#### Zugglied bei Fliessbeginn - Zustand $A1$

Setzt man die Fliessspannung an einem Ende des Zugglieds voraus, so folgen die Betonstahlspannungen im Zugglied zu:

In [21]:
# Stahlspannungen
sigma_1_s_com_A1 = f_sy_com_A
sigma_2_s_com_A1 = f_sy_com_A - 4 * tau_b0 * s_r / 2 / diam_x

put_out(symbolic=True, rows=1)

$$
\begin{aligned}
\sigma_{1 s , A1}& = f_{sy , A} = 570.0 \ \frac{\mathrm{N}}{\mathrm{mm}^{2}} \\ 
\sigma_{2 s , A1}& = f_{sy , A} - \frac{4 \cdot \tau_{b0} \cdot s_{r}}{2 \cdot \oslash_{x}} = 404.49 \ \frac{\mathrm{N}}{\mathrm{mm}^{2}} \end{aligned}
$$

Die folgenden Dehnungen stellen sich dabei ein:

In [22]:

# Stahldehnungen
eps_1_s_com_A1 = (sigma_1_s_com_A1 / E_s_com_A).to(un.permille)
eps_2_s_com_A1 = (sigma_2_s_com_A1 / E_s_com_A).to(un.permille)

put_out(symbolic=True, rows=1)

$$
\begin{aligned}
\varepsilon_{1 s , A1}& = \frac{\sigma_{1 s , A1}}{E_{s , A}} = 2.78 \ \mathrm{‰} \\ 
\varepsilon_{2 s , A1}& = \frac{\sigma_{2 s , A1}}{E_{s , A}} = 1.97 \ \mathrm{‰} \end{aligned}
$$

Über das gesamte Zugglied stellt sich die folgende mittlere Dehnung ein. Berechnungen nach @marti_tragverhalten_1999 Seite 156.

In [23]:
# Mittlere Stahldehnung nach 
eps_s_m_com_A1 = eps_sm_bilinear(E_s_com_A, E_sh_com_A, s_r, tau_b0, tau_b1, diam_x, f_sy_com_A, f_sy_com_A, f_su_com_A, eps_sy_com_A).to(un.permille)

put_out()


$$
\begin{aligned}
\varepsilon_{s m , A1}& = 2.38 \ \mathrm{‰} \quad &  \quad &  
 \end{aligned}
$$

Das Fliessmoment beträgt dabei:

In [24]:
# Fliessmoment
m_y_com_A = (a_s * f_sy_com_A * z_com_A).to(un.kNm / un.m)

put_out(symbolic=True)

$$
\begin{aligned}
m_{y , A}& = a_{s} \cdot f_{sy , A} \cdot z_{, A} = 110.25 \ \frac{\mathrm{kNm}}{\mathrm{m}} \quad &  \quad &  
 \end{aligned}
$$

Die @fig-jag_zustand_a1 zeigt das Verhalten entlang des Zugglieds.

In [25]:
# | echo: false


def plot_stress_strain(
    coordinates,
    filename,
    strain_mean=0,
    strain=False,
    stress=True,
):

    # Plot the polygon
    x, y = zip(*coordinates)  # Unzip the coordinates into x and y

    fig, ax = plt.subplots()

    if strain == True:
        stress = False
        y = np.array(y)*1000  # Convert to permille
        xlabel = r"$x$ [mm]"
        permille = '$10^{-3}$'
        ylabel = rf"$\varepsilon$ [{permille}]"
        
        # Append the first point to close the polygon
        x = np.append(x, x[0])
        y = np.append(y, y[0])

        # Plot the coordinates
        ax.plot(x, y, "black")
        ax.fill(x, y, color="darkgray", alpha=0.2)
        ax.axhline(strain_mean, linestyle='--', color = 'black')

    if stress == True:
        xlabel = r"$x$ [mm]"
        ylabel = r"$\sigma$ [N/mm$^2$]"
        
        # Append the first point to close the polygon
        x = np.append(x, x[0])
        y = np.append(y, y[0])

        # Plot the coordinates
        ax.plot(x, y, "black")


    # Add labels and show the plot
    ax.set_ylabel(ylabel)
    ax.set_xlabel(xlabel)

    # Save the plot to the specified file
    plt.savefig(filename)
    plt.close()


In [26]:
# | echo: false

coords_a1_stress = [
    (0, 0),
    (s_r.m / 2, 0),
    (s_r.m / 2, sigma_2_s_com_A1.m),
    (0, sigma_1_s_com_A1.m),
]

plot_stress_strain(coords_a1_stress, "imgs/jag_stress_a1.svg", stress=True)


coords_a1_strain = [
    (0, 0),
    (s_r.m / 2, 0),
    (s_r.m / 2, eps_2_s_com_A1),
    (0, eps_1_s_com_A1),
]

plot_stress_strain(coords_a1_strain, "imgs/jag_strain_a1.svg", strain=True,strain_mean=eps_s_m_com_A1.m)

![a) Spannungsverlauf b) Dehnungsverlauf innerhalb des halben Zugglieds des Zustands $A1$](../imgs/jag_A1_zugglied.svg){#fig-jag_zustand_a1}


#### Zugglied beim Erreichen der Zugfestigkeit - Zustand $A2$

Innerhalb des Zugglieds befindet sich der Betonstahl über der Fliessgrenze, sowohl bereichsweise unterhalb der Fliessgrenze. Dazu gilt es zunächst den plastischen Bereich zu bestimmen:

In [27]:
# Plastischer Bereich
Delta_x_pl_com_A2 = (f_su_com_A - f_sy_com_A) / (4 * tau_b1 / diam_x)

# Elastischer Bereich
Delta_x_el_com_A2 = s_r / 2 - Delta_x_pl_com_A2

put_out(symbolic=True, evaluate=False, rows=1)

$$
\begin{aligned}
\Delta_{x pl , A2}& = \frac{f_{su , A} - f_{sy , A}}{4 \cdot \tau_{b1} \cdot \frac{1}{\oslash_{x}}} = 36.25 \ \mathrm{mm} \\ 
\Delta_{x el , A2}& = \frac{s_{r}}{2} - \Delta_{x pl , A2} = 63.75 \ \mathrm{mm} \end{aligned}
$$

Setzt man die Fliessspannung zwischen dem elastischen und plastischem Bereich voraus, stellen sich die folgenden Betonstahlspannungen ein:

In [28]:
# Stahlspannungen
sigma_3_s_com_A2 = f_sy_com_A - 4 * tau_b0 * Delta_x_el_com_A2 / diam_x
sigma_2_s_com_A2 = f_sy_com_A
sigma_1_s_com_A2 = f_sy_com_A + 4 * tau_b1 * Delta_x_pl_com_A2 / diam_x

put_out(symbolic=True, rows=1)

$$
\begin{aligned}
\sigma_{3 s , A2}& = f_{sy , A} - \frac{4 \cdot \tau_{b0} \cdot \Delta_{x el , A2}}{\oslash_{x}} = 464.49 \ \frac{\mathrm{N}}{\mathrm{mm}^{2}} \\ 
\sigma_{2 s , A2}& = f_{sy , A} = 570.0 \ \frac{\mathrm{N}}{\mathrm{mm}^{2}} \\ 
\sigma_{1 s , A2}& = f_{sy , A} + \frac{4 \cdot \tau_{b1} \cdot \Delta_{x pl , A2}}{\oslash_{x}} = 600.0 \ \frac{\mathrm{N}}{\mathrm{mm}^{2}} \end{aligned}
$$

Die folgenden Dehnungen resultieren daraus:

In [29]:

# Stahldehnung
eps_1_s_com_A2 = eps_su_com_A
eps_2_s_com_A2 = eps_sy_com_A
eps_3_s_com_A2 = (sigma_3_s_com_A2 / E_s_com_A).to(un.permille)

put_out(symbolic=True, rows=1, evaluate=False)

$$
\begin{aligned}
\varepsilon_{1 s , A2}& = \varepsilon_{su , A} = 25 \ \mathrm{‰} \\ 
\varepsilon_{2 s , A2}& = \varepsilon_{sy , A} = 2.78 \ \mathrm{‰} \\ 
\varepsilon_{3 s , A2}& = \frac{\sigma_{3 s , A2}}{E_{s , A}} = 2.27 \ \mathrm{‰} \end{aligned}
$$

Und die mittlere Stahldehnung beträgt dabei:

In [30]:
# Mittlere Stahldehnung
eps_s_m_com_A2 = eps_sm_bilinear(
    E_s_com_A, E_sh_com_A, s_r, tau_b0, tau_b1, diam_x, f_su_com_A, f_sy_com_A, f_su_com_A, eps_sy_com_A
).to(un.permille)

put_out()

$$
\begin{aligned}
\varepsilon_{s m , A2}& = 6.64 \ \mathrm{‰} \quad &  \quad &  
 \end{aligned}
$$

Zusammengefasst ist das Spannungs- und Dehnungsverhalten für das halbe Zugglied in der @fig-jag_zustand_a2. 

In [31]:
# | echo: false

coords_A2_stress = [
    (0, 0), 
    (Delta_x_pl_com_A2.m, 0), 
    (s_r.m / 2, 0), 
    (s_r.m / 2, sigma_3_s_com_A2.m), 
    (Delta_x_pl_com_A2.m, sigma_2_s_com_A2.m),
    (0, sigma_1_s_com_A2.m)
    ]

plot_stress_strain(coords_A2_stress, "imgs/jag_stress_a2.svg", stress=True)


coords_Z2_strain = [
    (0, 0),
    (Delta_x_pl_com_A2.m, 0),
    (s_r.m / 2, 0),
    (s_r.m / 2, eps_3_s_com_A2),
    (Delta_x_pl_com_A2.m, eps_2_s_com_A2),
    (0, eps_1_s_com_A2),
]

plot_stress_strain(coords_Z2_strain, "imgs/jag_strain_a2.svg", strain=True, strain_mean=eps_s_m_com_A2.m)

![a) Spannungsverlauf b) Dehnungsverlauf innerhalb des halben Zugglieds des Zustands $A2$](../imgs/jag_A2_zugglied.svg){#fig-jag_zustand_a2}

#### Zusammenfassung

Die folgenden Resultate fassen die Grenzzustandsbetrachtung des Zugglieds ab:

In [32]:
eps_s_m_com_A1
m_y_com_A
eps_s_m_com_A2
m_u_com_A


put_out(symbolic=False, rows=2)

$$
\begin{aligned}
\varepsilon_{s m , A1}& = 2.38 \ \mathrm{‰} \quad & m_{y , A}& = 110.25 \ \frac{\mathrm{kNm}}{\mathrm{m}} \\ 
\varepsilon_{s m , A2}& = 6.64 \ \mathrm{‰} \quad & m_{u , A}& = 116.06 \ \frac{\mathrm{kNm}}{\mathrm{m}} \end{aligned}
$$

### Systemanalyse 

Die Systemanalyse gliedert sich in drei Zustände. Das System beim Fliessen des Mittelauflagers, das System beim Fliessbeginn im Feld und das System mit begrenzter plastischer Rotation über dem Mittelauflager. 

<!-- Das Systemverhalten der drei Zustände zeigt die @fig-jag_plast_rot_A.

![Definition der plastischen Rotation beim Mittelauflager. Links ein Fliessen beim Mittelauflager führt zu keiner plastischen Rotation, in der Mitte die maximale Rotation zum Erreichen der Traglast, und rechts die effektive Traglast mit begrenzter Plastischer Rotation](../imgs/jag_plast_rot_A.svg){#fig-jag_plast_rot_A} -->


#### Fliessen des Mittelauflagers - Zustand $SA1$

Setzt man beim Mittelauflager das Fliessmoment voraus, so stellen sich die folgenden Grössen ein.

In [33]:
m_SA1_com_B = m_y_com_A
# Die Streckenlast zum Erreichen des Fliessens bei B
q_SA1 = (8 * m_SA1_com_B*b / l**2).to(un.kN/un.m)


# maximale Verformung im Feld 
w_SA1 = (5 * q_SA1 * l**4 / (384 * EI_II_com_A) - m_y_com_A*b * l**2 / (16 * EI_II_com_A)).to(un.mm)

put_out(symbolic=True, evaluate=False, rows=1)

$$
\begin{aligned}
m_{SA1 , B}& = m_{y , A} = 110.25 \ \frac{\mathrm{kNm}}{\mathrm{m}} \\ 
q_{SA1}& = \frac{8 \cdot m_{SA1 , B} \cdot b}{l^{2}} = 13.78 \ \frac{\mathrm{kN}}{\mathrm{m}} \\ 
w_{SA1}& = \frac{5 \cdot q_{SA1} \cdot l^{4}}{384 \cdot EI_{II , A}} - \frac{m_{y , A} \cdot b \cdot l^{2}}{16 \cdot EI_{II , A}} = 36.28 \ \mathrm{mm} \end{aligned}
$$

#### Fliessbeginn im Feld - Zustand $SA2$

Setzt man beim Mittelauflager den Biegewiderstand voraus, sowie im Feld das Fliessmoment, stellen sich die folgenden Grössen ein. Illustriert ist dies in der @fig-jag_my_feld_A.

![Statisches System mit Biegemomenteneinwirkung beim Mittelauflager, sowie entsprechende Biegemomentenverteilung](../imgs/jag_my_feld_A.svg){#fig-jag_my_feld_A}

Daraus lässt sich die entsprechende Streckenlast ermitteln. Dies bedingt eine ausreichende Umlagerung, sprich Rotationsvermögen des Mittelauflagers.

In [34]:
# Streckenlast beim Fliessbeginn im Feld
q_SA2 = (((4 * m_y_com_A + 2 * m_u_com_A) / l**2 + 4 / l**2 * np.sqrt(m_y_com_A**2 + m_y_com_A * m_u_com_A))*b).to(un.kN/un.m)

put_out(symbolic=True, evaluate=False, rows=1)


$$
\begin{aligned}
q_{SA2}& = \left(\frac{4 \cdot m_{y , A} + 2 \cdot m_{u , A}}{l^{2}} + \frac{4 \cdot \sqrt{m_{y , A}^{2} + m_{y , A} \cdot m_{u , A}}}{l^{2}}\right) \cdot b = 20.39 \ \frac{\mathrm{kN}}{\mathrm{m}} \end{aligned}
$$

##### Kontrolle der Duktilität

Es gilt zu kontrollieren ob sich die vorausgesetzten Biegemomente einstellen können. Dazu ist der Rotationsbedarf mit dem Rotationsvermögen zu vergleichen. Der Bedarf ist der folgende.

In [35]:
# Rotationswinkel (Verwendung der Arbeitsgleichung)
alpha_SA2 = ((q_SA2 - q_SA1) * l**3 / (24 * EI_II_com_A) - (m_u_com_A - m_y_com_A) * b * l / (3 * EI_II_com_A)).to(un.deg)

# Verformungsbedarf
varphi_SA2_com_erf = 2 * alpha_SA2


put_out(symbolic=True, evaluate=False, rows=1)

$$
\begin{aligned}
\alpha_{SA2}& = \frac{\left(q_{SA2} - q_{SA1}\right) \cdot l^{3}}{24 \cdot EI_{II , A}} - \frac{\left(m_{u , A} - m_{y , A}\right) \cdot b \cdot l}{3 \cdot EI_{II , A}} = 0.89 \ \mathrm{°} \\ 
\varphi_{SA2 , erf}& = 2 \cdot \alpha_{SA2} = 1.77 \ \mathrm{°} \end{aligned}
$$

Mit den Näherungsformeln zur Bestimmung der plastischen Rotation im Gelenk, gemäss der @sec-abbruchkriterium, stellen sich die folgenden Grössen ein.

In [36]:
# Annahme für die Länge des Gelenks
l_pl = 2 * d_x

# Verformungsvermögen
varphi_max_com_A = (l_pl * (eps_s_m_com_A2 - eps_s_m_com_A1) / (d_x - x_u_com_A)).to(un.deg)

Delta_varphi_com_SA2 = varphi_max_com_A - varphi_SA2_com_erf 

put_out(symbolic=True, rows=1)

$$
\begin{aligned}
l_{pl}& = 2 \cdot d_{x} = 518.0 \ \mathrm{mm} \\ 
\varphi_{max , A}& = \frac{l_{pl} \cdot \left(\varepsilon_{s m , A2} - \varepsilon_{s m , A1}\right)}{d_{x} - x_{u , A}} = 0.53 \ \mathrm{°} \\ 
\Delta_{\varphi , SA2}& = \varphi_{max , A} - \varphi_{SA2 , erf} = -1.25 \ \mathrm{°} \end{aligned}
$$

Das Rotationsvermögen ist nicht ausreichend, das Tragwerk versagt bevor sich die berechnete Streckenlast einstellt.

#### Ermittlung der Traglast - Zustand $SA3$

Die Traglast ist somit sicherlich kleiner als die des Zustands $SA2$. Setzt man den Verdrehwinkel anhand des Verformungsvermögens voraus, so stellen sich die folgenden Grössen ein. Dem Zustand ist die Streckenlast aus dem Zustand $SA1$ zu überlagern.

In [37]:
# maximaler Rotationswinkel
alpha_SA3 = varphi_max_com_A / 2

# Streckenlast bis zum maximalen Rotationswinkel
Delta_q_com_SA3 = ((alpha_SA3 + (m_u_com_A - m_y_com_A) * l * b / (3 * EI_II_com_A)) * 24 * EI_II_com_A / l**3).to(un.kN / un.m)

# Traglast für B500A
q_SA3 = Delta_q_com_SA3 + q_SA1

put_out(symbolic=True, evaluate=False, rows=1)

$$
\begin{aligned}
\alpha_{SA3}& = \frac{\varphi_{max , A}}{2} = 0.26 \ \mathrm{°} \\ 
\Delta_{q , SA3}& = \left(\alpha_{SA3} + \frac{\left(m_{u , A} - m_{y , A}\right) \cdot l \cdot b}{3 \cdot EI_{II , A}}\right) \cdot 24 \cdot EI_{II , A} \cdot \frac{1}{l^{3}} = 2.47 \ \frac{\mathrm{kN}}{\mathrm{m}} \\ 
q_{SA3}& = \Delta_{q , SA3} + q_{SA1} = 16.25 \ \frac{\mathrm{kN}}{\mathrm{m}} \end{aligned}
$$

Dabei stellt sich im Feld ein Biegemoment ein, welches deutlich kleiner als das Fliessmoment ist.

In [38]:
expr = sp.sympify('(4* m_u  + 2 *m_u_apos)/l**2 + 4 / l**2 *sqrt(m_u**2+m_u*m_u_apos)-q')

m_SA3_analytical = sp.solve(expr, 'm_u')[0]


In [39]:

m_SA3_com_F = ((l**2*q_SA3 / 8 - m_u_com_A*b / 2 + (m_u_com_A*b)**2 / (2*l**2*q_SA3))/b).to(un.kNm/un.m)

put_out(symbolic=True, evaluate=False)


$$
\begin{aligned}
m_{SA3 , F}& = \frac{\frac{l^{2} \cdot q_{SA3}}{8} - \frac{m_{u , A} \cdot b}{2} + \frac{\left(m_{u , A} \cdot b\right)^{2}}{2 \cdot l^{2} \cdot q_{SA3}}}{b} = 78.45 \ \frac{\mathrm{kNm}}{\mathrm{m}} \quad &  \quad &  
 \end{aligned}
$$

Die dazugehörigen Felddurchbiegung zeigt die folgende Berechnung.

In [40]:
# maximale Verformung im Feld
w_SA3 = (5 * q_SA3 * l**4 / (384 * EI_II_com_A) - m_u_com_A * b * l**2 / (16 * EI_II_com_A)).to(un.mm)

put_out(symbolic=True, evaluate=False)

$$
\begin{aligned}
w_{SA3}& = \frac{5 \cdot q_{SA3} \cdot l^{4}}{384 \cdot EI_{II , A}} - \frac{m_{u , A} \cdot b \cdot l^{2}}{16 \cdot EI_{II , A}} = 49.65 \ \mathrm{mm} \quad &  \quad &  
 \end{aligned}
$$

#### Zusammenfassung

Die Grössen des zwei signifikanten Zustände sind hier erneut aufgelistet.

In [41]:
q_SA1
w_SA1
m_SA1_com_B

q_SA3
w_SA3
m_SA3_com_F


put_out(symbolic=False, rows = 3)

$$
\begin{aligned}
q_{SA1}& = 13.78 \ \frac{\mathrm{kN}}{\mathrm{m}} \quad & w_{SA1}& = 36.28 \ \mathrm{mm} \quad & m_{SA1 , B}& = 110.25 \ \frac{\mathrm{kNm}}{\mathrm{m}} \\ 
q_{SA3}& = 16.25 \ \frac{\mathrm{kN}}{\mathrm{m}} \quad & w_{SA3}& = 49.65 \ \mathrm{mm} \quad & m_{SA3 , F}& = 78.45 \ \frac{\mathrm{kNm}}{\mathrm{m}} \end{aligned}
$$

In [42]:
l_El = 0.1*un.m

# put_out()

In [43]:

# m_y_com_A
# m_u_com_A


# put_out(symbolic=True, rows = 1)

In [44]:
EI_II_com_A
chi_y_com_A = (m_y_com_A*b / EI_II_com_A).to(1/un.m)
chi_u_com_A = (eps_s_m_com_A2 / (d_x - x_u_com_A)).to(1/un.m)
# put_out(symbolic=True, rows = 1, precision=3)

In [45]:
momente_b500a = np.array([0, m_y_com_A.m, m_u_com_A.m])

chi_b500a = np.array([0, chi_y_com_A.m, chi_u_com_A.m])


fig_m_chi_b500a, ax_m_chi_b500a = plt.subplots()


ax_m_chi_b500a.plot(chi_b500a, momente_b500a, color='black')

# Add labels and show the plot
ax_m_chi_b500a.set_ylabel("$M$ [kNm]")
ax_m_chi_b500a.set_xlabel("$\chi$ [1/m]")

# Save the plot to the specified file
plt.savefig("imgs/jag_m_chi_b500a.svg")
plt.close()

In [46]:
varphi_y_com_A = (chi_y_com_A * (l_El/2)).to(un.rad)
varphi_u_com_A = (chi_u_com_A * l_El/2).to(un.rad)

# put_out(symbolic=True, precision = 5, rows = 1)



In [47]:
momente_b500a = np.array([0, m_y_com_A.m, m_u_com_A.m, m_u_com_A.m])

phi_b500a = np.array([0, varphi_y_com_A.m, varphi_u_com_A.m,varphi_u_com_A.m*1.5])


fig_m_phi_b500a, ax_m_phi_b500a = plt.subplots()


ax_m_phi_b500a.plot(phi_b500a, momente_b500a, color='black')

# Add labels and show the plot
ax_m_phi_b500a.set_ylabel("$M$ [kNm]")
ax_m_phi_b500a.set_xlabel("$\\varphi$ [rad]")

# Save the plot to the specified file
plt.savefig("imgs/jag_m_phi_b500a.svg")
plt.close()

In [48]:
varphi_max_com_A = varphi_max_com_A.to(un.rad)
# put_out(symbolic=False, rows = 1, precision=4)

In [49]:
n = (l_pl / l_El).to(un.los)

# put_out(symbolic=True, rows = 1, precision=0)

{{<pagebreak>}}

## Naturharter Betonstahl B500C

Die Berechnungen für des naturharten Betonstahl sind analog deren des kaltverformten Betonstahls. Zunächst sind die Eigenschaften des Betonstahls aufgelistet, danach folgt die Grenzzustandsanalyse des Zugglieds und abschliessend die Systemanalyse.

### Eigenschaften des Betonstahls

Die Eigenschaften des Betonstahls B500C sind mit einem Index $C$ gekennzeichnet.

In [50]:
f_sy_com_C = 500 * un.N / un.mm**2  # Fliessgrenze
f_su_com_C = 600 * un.N / un.mm**2  # Zugfestigkeit
eps_sy_com_C = 2.439 * un.permille  # Fliessdehnung
eps_sh_com_C = 20 * un.permille  # Dehnung bei Verfestigungsbeginn
eps_su_com_C = 125 * un.permille  # Bruchdehnung
E_s_com_C= 205 * un.kN / un.mm**2  # Elastizitätsmodul
E_sh_com_C = 952 * un.N / un.mm**2  # Verfestigungsmodul

put_out()

$$
\begin{aligned}
f_{sy , C}& = 500.0 \ \frac{\mathrm{N}}{\mathrm{mm}^{2}} \quad & f_{su , C}& = 600.0 \ \frac{\mathrm{N}}{\mathrm{mm}^{2}} \quad & \varepsilon_{sy , C}& = 2.44 \ \mathrm{‰} \\ 
\varepsilon_{sh , C}& = 20 \ \mathrm{‰} \quad & \varepsilon_{su , C}& = 125 \ \mathrm{‰} \quad & E_{s , C}& = 205.0 \ \frac{\mathrm{kN}}{\mathrm{mm}^{2}} \\ 
E_{sh , C}& = 952.0 \ \frac{\mathrm{N}}{\mathrm{mm}^{2}} \quad &  \quad &  
 \end{aligned}
$$

Die @fig-jag_stress_strain_b500c fasst die Eigenschaften zusammen.

In [51]:
# | echo: false

stress_C_array = np.array([0, f_sy_com_C.m, f_sy_com_C.m, f_su_com_C.m]) * un.MPa
strain_C_array = np.array([0, eps_sy_com_C, eps_sh_com_C, eps_su_com_C])


fig_b500c, ax_b500c = plt.subplots()

ax_b500c.plot(strain_C_array, stress_C_array.to(un.MPa).m, color="k")


# Set axis labels
ax_b500c.set_ylabel(r"$\sigma$ [N/mm$^2$]")
ax_b500c.set_xlabel(r"$\varepsilon_s$ [-]")

# Set limits
ax_b500c.set_xlim(xmax=135 / 1000)
ax_b500c.set_ylim(ymax=750)

# Show the plot
# plt.tight_layout()
plt.savefig("imgs/jag_stress_strain_b500c.svg")
plt.close()

### Gernzzustände des Zugglieds

Das Zugglied ist beim Fliessbeginn und beim Erreichen der Zugfestigkeit zu analysieren.

#### Zugglied bei Fliessbeginn - Zustand $C1$

Setzt man die Fliessspannung des Betonstahls beim Riss voraus, so resultieren die folgenden Betonstahlspannungen.

In [52]:
# Stahlspannungen
sigma_1_s_com_C1 = f_sy_com_C
sigma_2_s_com_C1 = f_sy_com_C - 4 * tau_b0 * s_r / 2 / diam_x

put_out(symbolic=True, rows = 1)

$$
\begin{aligned}
\sigma_{1 s , C1}& = f_{sy , C} = 500.0 \ \frac{\mathrm{N}}{\mathrm{mm}^{2}} \\ 
\sigma_{2 s , C1}& = f_{sy , C} - \frac{4 \cdot \tau_{b0} \cdot s_{r}}{2 \cdot \oslash_{x}} = 334.49 \ \frac{\mathrm{N}}{\mathrm{mm}^{2}} \end{aligned}
$$

Dabei stellen sich die folgenden Dehnungen ein:

In [53]:
# Stahldehnungen
eps_1_s_com_C1 = (sigma_1_s_com_C1 / E_s_com_C).to(un.permille)
eps_2_s_com_C1 = (sigma_2_s_com_C1 / E_s_com_C).to(un.permille)

put_out(symbolic=True, rows=1)

$$
\begin{aligned}
\varepsilon_{1 s , C1}& = \frac{\sigma_{1 s , C1}}{E_{s , C}} = 2.44 \ \mathrm{‰} \\ 
\varepsilon_{2 s , C1}& = \frac{\sigma_{2 s , C1}}{E_{s , C}} = 1.63 \ \mathrm{‰} \end{aligned}
$$

Nach @alvarez_einfluss_1998 Seite 163 lässt sich dich mittlere Dehnung für eine trilineare Betonstahlkennlinie bestimmen.

In [54]:
# Mittlere Stahldehnung beim Fliessen
eps_m_s_com_C1 = (eps_sm_bilinear(E_s_com_C, E_sh_com_C, s_r, tau_b0, tau_b1, diam_x, sigma_1_s_com_C1, f_sy_com_C, f_su_com_C, eps_sy_com_C)).to(un.permille)
put_out()

$$
\begin{aligned}
\varepsilon_{m s , C1}& = 2.04 \ \mathrm{‰} \quad &  \quad &  
 \end{aligned}
$$

Das Fliessmoment lässt sich folgend bestimmen.

In [55]:
# Fliessmoment
m_y_com_C = (a_s * f_sy_com_C * z_com_A).to(un.kNm/un.m)

put_out(symbolic=True)

$$
\begin{aligned}
m_{y , C}& = a_{s} \cdot f_{sy , C} \cdot z_{, A} = 96.71 \ \frac{\mathrm{kNm}}{\mathrm{m}} \quad &  \quad &  
 \end{aligned}
$$

Die @fig-jag_zustand_c1 zeigt das Spannungs- und Dehnungsverhalten entlang des halben Zugglieds.

In [56]:
# | echo: false

coords_c1_stress = [
    (0, 0),
    (s_r.m / 2, 0),
    (s_r.m / 2, sigma_2_s_com_C1.m),
    (0, sigma_1_s_com_C1.m),
]

plot_stress_strain(coords_c1_stress, "imgs/jag_stress_c1.svg", stress=True)


coords_c1_strain = [
    (0, 0),
    (s_r.m / 2, 0),
    (s_r.m / 2, eps_2_s_com_C1),
    (0, eps_1_s_com_C1),
]

plot_stress_strain(coords_c1_strain, "imgs/jag_strain_c1.svg", strain=True,strain_mean=eps_m_s_com_C1.m)

![a) Spannungsverlauf b) Dehnungsverlauf innerhalb des halben Zugglieds des Zustands $C1$](../imgs/jag_C1_zugglied.svg){#fig-jag_zustand_c1}

#### Zugglied beim Erreichen der Zugfestigkeit - Zustand $C2$

Setzt man dem Zugglied die Zugfestigkeit voraus, so verbleibt das gesamte Zugglied im plastischen Bereich. Es stellen sich die folgenden Spannungen ein.

In [57]:
# Stahlspannung zwischen den Rissen
sigma_1_s_com_C2 = f_su_com_C
sigma_2_s_com_C2 = f_su_com_C - 4 * tau_b1 / diam_x * s_r / 2

put_out(symbolic=True, rows =1)

$$
\begin{aligned}
\sigma_{1 s , C2}& = f_{su , C} = 600.0 \ \frac{\mathrm{N}}{\mathrm{mm}^{2}} \\ 
\sigma_{2 s , C2}& = f_{su , C} - \frac{4 \cdot \tau_{b1} \cdot s_{r}}{\oslash_{x} \cdot 2} = 517.24 \ \frac{\mathrm{N}}{\mathrm{mm}^{2}} \end{aligned}
$$

Die Dehnungen sind die folgenden.

In [58]:

# Stahldehnung zwischen den Rissen
eps_1_s_com_C2 = ((sigma_2_s_com_C2 - f_sy_com_C) / E_sh_com_C + eps_sh_com_C).to(un.permille)
eps_2_s_com_C2 = eps_su_com_C

put_out(symbolic=True, rows=1)

$$
\begin{aligned}
\varepsilon_{1 s , C2}& = \frac{\sigma_{2 s , C2} - f_{sy , C}}{E_{sh , C}} + \varepsilon_{sh , C} = 38.11 \ \mathrm{‰} \\ 
\varepsilon_{2 s , C2}& = \varepsilon_{su , C} = 125 \ \mathrm{‰} \end{aligned}
$$

Und die mittlere Stahldehnung lässt sich ebenfalls bestimmen.

In [59]:

eps_m_s_com_C2 = (eps_1_s_com_C2 + eps_2_s_com_C2) / 2

put_out()

$$
\begin{aligned}
\varepsilon_{m s , C2}& = 81.56 \ \mathrm{‰} \quad &  \quad &  
 \end{aligned}
$$

Abschliessend ist zu kontrollieren, ob der Betonstahl die Zugfestigkeit erreichen kann, oder sich ein frühzeitiges Betondruckversagen einstellt. Die mittlere Betonstahldehnung führt zur folgenden Betonstauchung, welche grösser als die Bruchstauchung ist. 

In [60]:
# Betonstauchung
eps_c_com_C2 = eps_m_s_com_C2 / (d_x - x_u_com_A) * x_u_com_A

eps_cu

put_out(symbolic=True)

$$
\begin{aligned}
\varepsilon_{c , C2}& = \frac{\varepsilon_{m s , C2} \cdot x_{u , A}}{d_{x} - x_{u , A}} = 6.13 \ \mathrm{‰} \quad & \varepsilon_{cu}& = 5 \ \mathrm{‰} \quad &  
 \end{aligned}
$$

Die mittlere Betonstahldehung ist folglich anhand der begrenzten Betonstauchung zu berechnen. 

In [61]:
eps_m_s_com_adm = eps_cu * (d_x - x_u_com_A)/x_u_com_A
put_out(symbolic=True)


$$
\begin{aligned}
\varepsilon_{m s , adm}& = \frac{\varepsilon_{cu} \cdot \left(d_{x} - x_{u , A}\right)}{x_{u , A}} = 66.51 \ \mathrm{‰} \quad &  \quad &  
 \end{aligned}
$$

Damit folgen die Dehnungen für diesen Zustand zu:

In [62]:
# Stahldehnung zwischen den Rissen
eps_1_s_com_C2 = eps_m_s_com_adm + tau_b1 * 4* s_r/4 / (diam_x*E_sh_com_C)
eps_2_s_com_C2 = eps_m_s_com_adm - tau_b1 * 4 * s_r/4/ (diam_x*E_sh_com_C)

put_out(symbolic=True, rows = 1, evaluate=True)

$$
\begin{aligned}
\varepsilon_{1 s , C2}& = \varepsilon_{m s , adm} + \frac{s_{r} \cdot \tau_{b1}}{E_{sh , C} \cdot \oslash_{x}} = 109.97 \ \mathrm{‰} \\ 
\varepsilon_{2 s , C2}& = \varepsilon_{m s , adm} - \frac{s_{r} \cdot \tau_{b1}}{E_{sh , C} \cdot \oslash_{x}} = 23.04 \ \mathrm{‰} \end{aligned}
$$

Und daraus folgen die Spannungen.

In [63]:
# Stahlspannung zwischen den Rissen
sigma_1_s_com_C2 = f_sy_com_C + (eps_1_s_com_C2 - eps_sh_com_C)*E_sh_com_C
sigma_2_s_com_C2 = f_sy_com_C + (eps_2_s_com_C2 - eps_sh_com_C)*E_sh_com_C


put_out(symbolic=True, rows=1)

$$
\begin{aligned}
\sigma_{1 s , C2}& = f_{sy , C} + \left(\varepsilon_{1 s , C2} - \varepsilon_{sh , C}\right) \cdot E_{sh , C} = 585.65 \ \frac{\mathrm{N}}{\mathrm{mm}^{2}} \\ 
\sigma_{2 s , C2}& = f_{sy , C} + \left(\varepsilon_{2 s , C2} - \varepsilon_{sh , C}\right) \cdot E_{sh , C} = 502.9 \ \frac{\mathrm{N}}{\mathrm{mm}^{2}} \end{aligned}
$$

Dabei folgt der Biegewiderstand, begrenzt durch ein Betondruckversagen, zu:

In [64]:
m_u_com_C = (a_s * sigma_1_s_com_C2 * z_com_A).to(un.kNm/un.m)

put_out(symbolic=True)

$$
\begin{aligned}
m_{u , C}& = a_{s} \cdot \sigma_{1 s , C2} \cdot z_{, A} = 113.28 \ \frac{\mathrm{kNm}}{\mathrm{m}} \quad &  \quad &  
 \end{aligned}
$$

In der @fig-jag_zustand_c2 ist das Spannungs- und Dehnungs-Verhalten entlang des Zugglieds aufgezeigt.

In [65]:
# | echo: false

coords_c2_stress = [
    (0, 0),
    (s_r.m / 2, 0),
    (s_r.m / 2, sigma_2_s_com_C2.m),
    (0, sigma_1_s_com_C2.m),
]

plot_stress_strain(coords_c2_stress, "imgs/jag_stress_c2.svg", stress=True)


coords_c2_strain = [
    (0, 0),
    (s_r.m / 2, 0),
    (s_r.m / 2, eps_2_s_com_C2),
    (0, eps_1_s_com_C2),
]

plot_stress_strain(coords_c2_strain, "imgs/jag_strain_c2.svg", strain=True,strain_mean=eps_m_s_com_adm.m)

![a) Spannungsverlauf b) Dehnungsverlauf innerhalb des halben Zugglieds des Zustands $C2$](../imgs/jag_C2_zugglied.svg){#fig-jag_zustand_c2}

#### Zusammenfassung


Eine Zusammenfassung der berechneten Grössen bildet den Abschluss der Grenzzustandsbetrachtung des Zugglieds.

In [66]:
eps_m_s_com_C1
m_y_com_C

eps_m_s_com_adm
m_u_com_C

put_out(rows = 2)


$$
\begin{aligned}
\varepsilon_{m s , C1}& = 2.04 \ \mathrm{‰} \quad & m_{y , C}& = 96.71 \ \frac{\mathrm{kNm}}{\mathrm{m}} \\ 
\varepsilon_{m s , adm}& = 66.51 \ \mathrm{‰} \quad & m_{u , C}& = 113.28 \ \frac{\mathrm{kNm}}{\mathrm{m}} \end{aligned}
$$

### Systemanalyse

Nach der Grenzzustandsanalyse des Zugglieds folgt die Systemanalyse. Die Systemanalyse betrachtet das System beim Fliessen des Mittelauflagers, beim Fliessbeginn im Feld und bei begrenzter Rotation beim Mittelauflager.

#### Fliessen des Mittelauflager - Zustand $SC1$

Setzt man das Fliessmoment beim Mittelauflager voraus, so stellen sich die folgenden Grössen ein.

In [67]:
m_SC1_com_B = m_y_com_C

# Streckenlast beim Fliessen des Mittelauflagers
q_SC1 = (8 * m_SC1_com_B / l**2 * b).to(un.kN/un.m)



# Verformung beim Fliessen des Mittelauflagers
w_SC1 = (5 * q_SC1 * l**4 / (384 * EI_II_com_A) - m_SC1_com_B * b * l**2 / (16 * EI_II_com_A)).to(un.mm)

put_out(symbolic=True, rows=1)

$$
\begin{aligned}
m_{SC1 , B}& = m_{y , C} = 96.71 \ \frac{\mathrm{kNm}}{\mathrm{m}} \\ 
q_{SC1}& = \frac{8 \cdot m_{SC1 , B} \cdot b}{l^{2}} = 12.09 \ \frac{\mathrm{kN}}{\mathrm{m}} \\ 
w_{SC1}& = \frac{5 \cdot q_{SC1} \cdot l^{4}}{384 \cdot EI_{II , A}} - \frac{m_{SC1 , B} \cdot b \cdot l^{2}}{16 \cdot EI_{II , A}} = 31.82 \ \mathrm{mm} \end{aligned}
$$

#### Fliessbeginn im Feld - Zustand $SC2$

Das Fliessmoment ist im Feld vorausgesetzt, das Biegemoment beim Auflager gilt dabei als unbekannt, ist jedoch sicherlich grösser als das Fliessmoment. 

Durch einen iterativen Prozess ist das Biegemoment beim Auflager zu bestimmen. Dazu ist eine Betonstahlspannung im Zugglied vorauszusetzen, anhand dieser ist das Rotationsvermögen und das Biegemoment beim Mitterauflager zu bestimmen. Für den gewählten Zustand lässt sich der Rotationsbedarf berechnen. Decken sich das Rotationsvermögen und der Rotationsbedarf, so ist die passende Betonstahlspannung im Zugglied gefunden. Welche somit auch die Zustandslinien der Biegemomente definiert. 

Das Feldmoment entspricht somit der folgenden Gleichung.

In [68]:
m_SC2_com_F = m_y_com_C

put_out(symbolic=True)

$$
\begin{aligned}
m_{SC2 , F}& = m_{y , C} = 96.71 \ \frac{\mathrm{kNm}}{\mathrm{m}} \quad &  \quad &  
 \end{aligned}
$$

Nun folgt die Analyse des Zugglieds. Es gilt die Annahme, dass sich ein elastischer und plastischer Bereich einstellt. Die folgende Berechnung zeigt die Abmessungen des elastischen und plastischen Bereichs im halben Zugglied.

In [69]:
# Fliessspannung wird vorausgesetzt
Delta_x_pl_com_SC2 = Delta_x_pl_com_A2
Delta_x_el_com_SC2 = s_r / 2 - Delta_x_pl_com_SC2

put_out(symbolic=True, rows = 1)



$$
\begin{aligned}
\Delta_{x pl , SC2}& = \Delta_{x pl , A2} = 36.25 \ \mathrm{mm} \\ 
\Delta_{x el , SC2}& = \frac{s_{r}}{2} - \Delta_{x pl , SC2} = 63.75 \ \mathrm{mm} \end{aligned}
$$

Die hier eingeführte Betonstahlspannung im Übergang zwischen elastischem und plastischem Bereich ist die zu iterierende Grösse. Die weiteren Spannungen ergeben sich anhand der Schubspannungs-Schlupf-Beziehung.

In [70]:
sigma_2_s_com_SC2 = 500 * un.N / un.mm**2
sigma_1_s_com_SC2 = sigma_2_s_com_SC2 + 4 * tau_b1 / diam_x * Delta_x_pl_com_SC2
sigma_3_s_com_SC2 = sigma_2_s_com_SC2 - 4 * tau_b0 / diam_x * Delta_x_el_com_SC2

put_out(symbolic=True, rows=1)

$$
\begin{aligned}
\sigma_{2 s , SC2}& = 500.0 \ \frac{\mathrm{N}}{\mathrm{mm}^{2}} \\ 
\sigma_{1 s , SC2}& = \sigma_{2 s , SC2} + \frac{4 \cdot \tau_{b1} \cdot \Delta_{x pl , SC2}}{\oslash_{x}} = 530.0 \ \frac{\mathrm{N}}{\mathrm{mm}^{2}} \\ 
\sigma_{3 s , SC2}& = \sigma_{2 s , SC2} - \frac{4 \cdot \tau_{b0} \cdot \Delta_{x el , SC2}}{\oslash_{x}} = 394.49 \ \frac{\mathrm{N}}{\mathrm{mm}^{2}} \end{aligned}
$$

Dabei stellen sich die folgenden Dehnungen ein.

In [71]:
# Dehnungen
eps_1_s_com_SC2 = ((sigma_1_s_com_SC2 - sigma_2_s_com_SC2) / E_sh_com_C + eps_sh_com_C).to(un.permille)
eps_21_s_com_SC2 = eps_sh_com_C
eps_22_s_com_SC2 = eps_sy_com_A
eps_3_s_com_SC2 = (sigma_3_s_com_SC2 / E_s_com_A).to(un.permille)

put_out(symbolic=True, rows=1)

$$
\begin{aligned}
\varepsilon_{1 s , SC2}& = \frac{\sigma_{1 s , SC2} - \sigma_{2 s , SC2}}{E_{sh , C}} + \varepsilon_{sh , C} = 51.51 \ \mathrm{‰} \\ 
\varepsilon_{21 s , SC2}& = \varepsilon_{sh , C} = 20 \ \mathrm{‰} \\ 
\varepsilon_{22 s , SC2}& = \varepsilon_{sy , A} = 2.78 \ \mathrm{‰} \\ 
\varepsilon_{3 s , SC2}& = \frac{\sigma_{3 s , SC2}}{E_{s , A}} = 1.92 \ \mathrm{‰} \end{aligned}
$$

Abschliessend lässt sich mit der trilinearen Spannungs-Dehnungs-Beziehung die Mittlere Dehnung bestimmen. In @alvarez_einfluss_1998 Seite 163 ist eine analytische Lösung dazu aufgezeigt.

In [72]:
eps_m_s_com_SC2 = 14.33*un.permille
put_out(rows=1)

$$
\begin{aligned}
\varepsilon_{m s , SC2}& = 14.33 \ \mathrm{‰} \end{aligned}
$$

Dargestellt sind die Resultate in der @fig-jag_zustand_sc2.

In [73]:
# | echo: false

coords_SC2_stress = [
    (0, 0), 
    (Delta_x_pl_com_SC2.m, 0), 
    (s_r.m / 2, 0), 
    (s_r.m / 2, sigma_3_s_com_SC2.m), 
    (Delta_x_pl_com_SC2.m, sigma_2_s_com_SC2.m),
    (0, sigma_1_s_com_SC2.m)
    ]

plot_stress_strain(coords_SC2_stress, "imgs/jag_stress_sc2.svg", stress=True)


coords_SC2_strain = [
    (0, 0),
    (Delta_x_pl_com_SC2.m, 0),
    (s_r.m / 2, 0),
    (s_r.m / 2, eps_3_s_com_SC2),
    (Delta_x_pl_com_SC2.m, eps_22_s_com_SC2),
    (Delta_x_pl_com_SC2.m, eps_21_s_com_SC2),
    (0, eps_1_s_com_SC2),
]

plot_stress_strain(coords_SC2_strain, "imgs/jag_strain_sc2.svg", strain=True, strain_mean=eps_m_s_com_SC2.m)

![a) Spannungsverlauf b) Dehnungsverlauf innerhalb des halben Zugglieds des Zustands $SC2$](../imgs/jag_SC2_zugglied.svg){#fig-jag_zustand_sc2}

Das Rotationsvermögen lässt sich anhand der mittleren Dehnungen bestimmen.

In [74]:
varphi_SC2_com_pl = (l_pl * (eps_m_s_com_SC2 - eps_m_s_com_C1) / (d_x - x_u_com_A)).to(un.deg)

put_out(symbolic=True)

$$
\begin{aligned}
\varphi_{SC2 , pl}& = \frac{l_{pl} \cdot \left(\varepsilon_{m s , SC2} - \varepsilon_{m s , C1}\right)}{d_{x} - x_{u , A}} = 1.51 \ \mathrm{°} \quad &  \quad &  
 \end{aligned}
$$

Mit der gewählten Betonstahlspannung lässt sich das Biegemoment über dem Auflager bestimmen.

In [75]:
# Biegemoment resultiert daraus zu
m_SC2_com_B = (a_s * sigma_1_s_com_SC2 * z_com_A).to(un.kNm/un.m)


put_out(symbolic=True)

$$
\begin{aligned}
m_{SC2 , B}& = a_{s} \cdot \sigma_{1 s , SC2} \cdot z_{, A} = 102.52 \ \frac{\mathrm{kNm}}{\mathrm{m}} \quad &  \quad &  
 \end{aligned}
$$

Dabei stellt sich die folgende Streckenlast für den Zustand ein.

In [76]:

# Streckenlast Fliessen im Feld, verfestigend über dem Auflager
q_SC2 = (((4 * m_y_com_C + 2 * m_SC2_com_F) / l**2 + 4 / l**2 * np.sqrt(m_SC2_com_F**2 + m_SC2_com_F * m_SC2_com_B)) * b).to(un.kN/un.m)
put_out(symbolic=True, rows=1)

$$
\begin{aligned}
q_{SC2}& = \left(\frac{4 \cdot m_{y , C} + 2 \cdot m_{SC2 , F}}{l^{2}} + \frac{4 \cdot \sqrt{m_{SC2 , F}^{2} + m_{SC2 , F} \cdot m_{SC2 , B}}}{l^{2}}\right) \cdot b = 17.74 \ \frac{\mathrm{kN}}{\mathrm{m}} \end{aligned}
$$

Abschliessend ist der Rotationsbedarf zu bestimmen und mit dem Rotationsvermögen zu vergleichen.

In [77]:
# Kontrolle des Verformungsbedarfs
alpha_SC2 = ((q_SC2 - q_SC1) * l**3 / (24 * EI_II_com_A)- (m_SC2_com_B - m_SC2_com_F)  * l*b / (3 * EI_II_com_A)).to(un.deg)

# Erforderliche Rotation
varphi_SC2_com_erf = 2 * alpha_SC2

# Kontrolle ob das Verformungsvermögen gleich dem Verformungsbedarf ist
Delta_varphi_SC3 = varphi_SC2_com_erf - varphi_SC2_com_pl

put_out(symbolic=True, rows=1)

$$
\begin{aligned}
\alpha_{SC2}& = \frac{\left(q_{SC2} - q_{SC1}\right) \cdot l^{3}}{24 \cdot EI_{II , A}} - \frac{\left(m_{SC2 , B} - m_{SC2 , F}\right) \cdot l \cdot b}{3 \cdot EI_{II , A}} = 0.74 \ \mathrm{°} \\ 
\varphi_{SC2 , erf}& = 2 \cdot \alpha_{SC2} = 1.49 \ \mathrm{°} \\ 
\Delta_{\varphi SC3}& = \varphi_{SC2 , erf} - \varphi_{SC2 , pl} = -0.03 \ \mathrm{°} \end{aligned}
$$

Die Differenz zwischen dem Rotationsvermögen und dem Rotationsbedarf ist vernachlässigbar klein. Der Zustand ist somit zulässig. Abschliessend lässt sich die Feldverformung bestimmen.

In [78]:
# Die Mittendurchbiegung beträgt
w_SC2 = (5 * q_SC2 * l**4 / (384 * EI_II_com_A) - m_SC2_com_B * b * l**2 / (16 * EI_II_com_A)).to(un.mm)

put_out(symbolic=True)

$$
\begin{aligned}
w_{SC2}& = \frac{5 \cdot q_{SC2} \cdot l^{4}}{384 \cdot EI_{II , A}} - \frac{m_{SC2 , B} \cdot b \cdot l^{2}}{16 \cdot EI_{II , A}} = 66.16 \ \mathrm{mm} \quad &  \quad &  
 \end{aligned}
$$

#### Ermittlung der Traglast - Zustand $SC3$

Das Rotationsvermögen ist noch nicht vollständig ausgeschöpft, es sind weitere Tragreserven vorhanden. Diese gilt es im Zustand $SC3$ zu mobilisieren. Dazu ist zunächst das maximale Rotationsvermögen zu bestimmen.


In [79]:
# Plastischer Rotationswinkel, Rotationsbedarf für vollständige Umlagerung.
varphi_max_com_C = (l_pl * (eps_m_s_com_adm - eps_m_s_com_C1) / (d_x - x_u_com_A)).to(un.deg)

put_out(symbolic=True)

$$
\begin{aligned}
\varphi_{max , C}& = \frac{l_{pl} \cdot \left(\varepsilon_{m s , adm} - \varepsilon_{m s , C1}\right)}{d_{x} - x_{u , A}} = 7.94 \ \mathrm{°} \quad &  \quad &  
 \end{aligned}
$$

Beim Mittelauflager ist der Biegewiderstand vorausgesetzt. Im Feld stellt sich ebenfalls ein plastisches Gelenk ein. Dabei ist anzunehmen, dass das Feldmoment kleiner als der Biegewiderstand ist. Hier gilt es wiederum einen passenden Spannungszustand im Zugglied zu finden, welcher ein mit dem Rotationsvermögen verträgliches System zur Folge hat. Dazu sind die folgenden Dehnungen im Zugglied im Feld vorausgesetzt.

In [80]:
eps_2_s_com_SC3 = eps_sh_com_C
eps_1_s_com_SC3 = eps_2_s_com_SC3 + 4 * tau_b1 * s_r / 2 / (diam_x * E_sh_com_C)


put_out(symbolic=True, rows=1)

$$
\begin{aligned}
\varepsilon_{2 s , SC3}& = \varepsilon_{sh , C} = 20 \ \mathrm{‰} \\ 
\varepsilon_{1 s , SC3}& = \varepsilon_{2 s , SC3} + \frac{4 \cdot \tau_{b1} \cdot s_{r}}{2 \cdot \oslash_{x} \cdot E_{sh , C}} = 106.93 \ \mathrm{‰} \end{aligned}
$$

Diese resultieren in den folgenden Spannungen.

In [81]:

sigma_1_s_com_SC3 = ((eps_1_s_com_SC3 - eps_2_s_com_SC3) * E_sh_com_C + f_sy_com_C).to(un.N/un.mm**2)
sigma_2_s_com_SC3 = (f_sy_com_C).to(un.N/un.mm**2)
put_out(symbolic=True, rows=1)

$$
\begin{aligned}
\sigma_{1 s , SC3}& = \left(\varepsilon_{1 s , SC3} - \varepsilon_{2 s , SC3}\right) \cdot E_{sh , C} + f_{sy , C} = 582.76 \ \frac{\mathrm{N}}{\mathrm{mm}^{2}} \\ 
\sigma_{2 s , SC3}& = f_{sy , C} = 500.0 \ \frac{\mathrm{N}}{\mathrm{mm}^{2}} \end{aligned}
$$

Daraus folgt die mittlere Dehnung.

In [82]:
eps_m_s_com_SC3 = (eps_2_s_com_SC3 + eps_1_s_com_SC3) / 2

put_out(symbolic=True)

$$
\begin{aligned}
\varepsilon_{m s , SC3}& = \frac{\varepsilon_{2 s , SC3} + \varepsilon_{1 s , SC3}}{2} = 63.46 \ \mathrm{‰} \quad &  \quad &  
 \end{aligned}
$$

Dargestellt ist der Spannungs- und Dehnungszustand des halben Zugglieds in der @fig-jag_zustand_sc3.

In [83]:
# | echo: false

coords_sc3_stress = [
    (0, 0),
    (s_r.m / 2, 0),
    (s_r.m / 2, sigma_2_s_com_SC3.m),
    (0, sigma_1_s_com_SC3.m),
]

plot_stress_strain(coords_sc3_stress, "imgs/jag_stress_sc3.svg", stress=True)


coords_sc3_strain = [
    (0, 0),
    (s_r.m / 2, 0),
    (s_r.m / 2, eps_2_s_com_SC3),
    (0, eps_1_s_com_SC3),
]

plot_stress_strain(coords_sc3_strain, "imgs/jag_strain_sc3.svg", strain=True, strain_mean=eps_m_s_com_SC3.m)

![a) Spannungsverlauf b) Dehnungsverlauf innerhalb des halben Zugglieds des Zustands $SC3$](../imgs/jag_sc3_zugglied.svg){#fig-jag_zustand_sc3}

Das Biegemoment im Feld und beim Mittelauflager sind folgend aufgezeigt.

In [84]:
m_SC3_com_F = (a_s * sigma_1_s_com_SC3 * z_com_A).to(un.kNm/un.m)

m_SC3_com_B = m_u_com_C
put_out(symbolic=True, rows = 1)


$$
\begin{aligned}
m_{SC3 , F}& = a_{s} \cdot \sigma_{1 s , SC3} \cdot z_{, A} = 112.72 \ \frac{\mathrm{kNm}}{\mathrm{m}} \\ 
m_{SC3 , B}& = m_{u , C} = 113.28 \ \frac{\mathrm{kNm}}{\mathrm{m}} \end{aligned}
$$

Für diesen Zustand stellt sich die folgende Streckenlast ein.

In [85]:

# Streckenlast Fliessen im Feld, verfestigend über dem Auflager
q_SC3 = (((4 * m_SC3_com_F + 2 * m_u_com_C) / l**2 + 4 / l**2 * np.sqrt(m_SC3_com_F**2 + m_SC3_com_F * m_u_com_C)) * b).to(un.kN/un.m)

put_out(symbolic=True)


$$
\begin{aligned}
q_{SC3}& = \left(\frac{4 \cdot m_{SC3 , F} + 2 \cdot m_{u , C}}{l^{2}} + \frac{4 \cdot \sqrt{m_{SC3 , F}^{2} + m_{SC3 , F} \cdot m_{u , C}}}{l^{2}}\right) \cdot b = 20.56 \ \frac{\mathrm{kN}}{\mathrm{m}} \quad &  \quad &  
 \end{aligned}
$$

Abschliessend ist die plastische Rotation im Feld zu kontrollieren. Die Lage des plastischen Gelenks im Feld ist hierbei abgeschätzt. Dabei entspricht $w$ der plastischen Rotation, welche kleiner als das Rotationsvermögen sein muss. Illustriert ist dies in der @fig-jag_plast_feld_C.

![Annahme für die Lage des Gelenks im Feld](../imgs/jag_plast_feld_C.svg){#fig-jag_plast_feld_C}

Mit der gewählten Spannungsverteilung im Zugglied stellt sich die folgende Feldrotation ein.

In [86]:

# Plastische rotation im Feld
varphi_SC3_com_F = (l_pl * (eps_m_s_com_SC3 - eps_m_s_com_C1) / (d_x - x_u_com_A)).to(un.deg)

put_out(symbolic=True)

$$
\begin{aligned}
\varphi_{SC3 , F}& = \frac{l_{pl} \cdot \left(\varepsilon_{m s , SC3} - \varepsilon_{m s , C1}\right)}{d_{x} - x_{u , A}} = 7.57 \ \mathrm{°} \quad &  \quad &  
 \end{aligned}
$$

Und beim Mittelauflager muss die folgende Rotation verfügbar sein:

In [87]:
# Kontrolle des Verformungsbedarfs
alpha_SC3 = (q_SC3 * l**3 / (24 * EI_II_com_A)- (m_u_com_C) * b * l / (3 * EI_II_com_A)+ 0.414 * varphi_SC3_com_F).to(un.deg)

# Erforderliche Rotation
varphi_SC3_com_erf = 2 * alpha_SC3


put_out(symbolic=True, rows = 1)

$$
\begin{aligned}
\alpha_{SC3}& = \frac{q_{SC3} \cdot l^{3}}{24 \cdot EI_{II , A}} - \frac{m_{u , C} \cdot b \cdot l}{3 \cdot EI_{II , A}} + 0.414 \cdot \varphi_{SC3 , F} = 4.1 \ \mathrm{°} \\ 
\varphi_{SC3 , erf}& = 2 \cdot \alpha_{SC3} = 8.2 \ \mathrm{°} \end{aligned}
$$

Welche mit der maximalen plastischen Rotation zu vergleichen ist.

In [88]:
# Kontrolle ob das Verformungsvermögen gleich dem Verformungsbedarf ist

varphi_max_com_C
Delta_varphi_SC3 = varphi_SC3_com_erf - varphi_max_com_C

put_out(symbolic=True, rows = 1)

$$
\begin{aligned}
\varphi_{max , C}& = \frac{l_{pl} \cdot \left(\varepsilon_{m s , adm} - \varepsilon_{m s , C1}\right)}{d_{x} - x_{u , A}} = 7.94 \ \mathrm{°} \\ 
\Delta_{\varphi SC3}& = \varphi_{SC3 , erf} - \varphi_{max , C} = 0.25 \ \mathrm{°} \end{aligned}
$$

Die erforderliche Rotation des Mittelauflagers ist in etwa gleich dem Rotationsvermögen. Es ist ein gültiger Zustand gefunden. Abschliessend lässt sich die Feldverformung bestimmen.

In [89]:
# Die Mittendurchbiegung beträgt
w_SC3 = (5 * q_SC3 * l**4 / (384 * EI_II_com_A)- m_u_com_C * b * l**2 / (16 * EI_II_com_A)+ 0.414 * varphi_SC3_com_F * l / 2).to(un.mm)

put_out(symbolic=True, evaluate=False)

$$
\begin{aligned}
w_{SC3}& = \frac{5 \cdot q_{SC3} \cdot l^{4}}{384 \cdot EI_{II , A}} - \frac{m_{u , C} \cdot b \cdot l^{2}}{16 \cdot EI_{II , A}} + \frac{0.414 \cdot \varphi_{SC3 , F} \cdot l}{2} = 298.14 \ \mathrm{mm} \quad &  \quad &  
 \end{aligned}
$$

#### Zusammenfassung

Der Abschluss der Berechnung des naturharten Stahls bildet die Zusammenstellung der berechneten Resultate.

In [90]:
q_SC1
w_SC1
m_SC1_com_B

q_SC2
w_SC2
m_SC2_com_B


q_SC3
w_SC3
m_SC3_com_F



put_out()

$$
\begin{aligned}
q_{SC1}& = 12.09 \ \frac{\mathrm{kN}}{\mathrm{m}} \quad & w_{SC1}& = 31.82 \ \mathrm{mm} \quad & m_{SC1 , B}& = 96.71 \ \frac{\mathrm{kNm}}{\mathrm{m}} \\ 
q_{SC2}& = 17.74 \ \frac{\mathrm{kN}}{\mathrm{m}} \quad & w_{SC2}& = 66.16 \ \mathrm{mm} \quad & m_{SC2 , B}& = 102.52 \ \frac{\mathrm{kNm}}{\mathrm{m}} \\ 
q_{SC3}& = 20.56 \ \frac{\mathrm{kN}}{\mathrm{m}} \quad & w_{SC3}& = 298.14 \ \mathrm{mm} \quad & m_{SC3 , F}& = 112.72 \ \frac{\mathrm{kNm}}{\mathrm{m}} \end{aligned}
$$

In [91]:

# m_y_com_C
# m_u_com_C

# put_out(symbolic=True, rows = 1)

In [92]:
chi_y_com_C = (m_y_com_C*b / EI_II_com_A).to(1/un.m)

# put_out(symbolic=True, rows=1, precision=4)

In [93]:
chi_sh_com_C = (eps_sh_com_C / (d_x - x_u_com_A)).to(1/un.m)
# put_out(symbolic=True, rows=1, precision=4)

In [94]:
chi_u_com_C = (eps_m_s_com_adm / (d_x - x_u_com_A)).to(1/un.m)

# put_out(symbolic=True, rows=1, precision=4)

In [95]:
momente_b500c = np.array([0, m_y_com_C.m, m_y_com_C.m, m_u_com_C.m])

chi_b500c = np.array([0, chi_y_com_C.m, chi_sh_com_C.m, chi_u_com_C.m])


fig_m_chi_b500c, ax_m_chi_b500c = plt.subplots()


ax_m_chi_b500c.plot(chi_b500c, momente_b500c, color='black')

# Add labels and show the plot
ax_m_chi_b500c.set_ylabel("$M$ [kNm]")
ax_m_chi_b500c.set_xlabel("$\chi$ [1/m]")

# Save the plot to the specified file
plt.savefig("imgs/jag_m_chi_b500c.svg")
plt.close()

In [96]:
varphi_y_com_C = (chi_y_com_C * l_El/2).to(un.rad)
varphi_sh_com_C = (chi_sh_com_C * l_El/2).to(un.rad)
varphi_u_com_C = (chi_u_com_C * l_El/2).to(un.rad)
# put_out(symbolic=True, precision = 4, rows = 1)

In [97]:
momente_b500c = np.array([0, m_y_com_C.m, m_y_com_C.m, m_u_com_C.m, m_u_com_C.m])

phi_b500c = np.array([0, varphi_y_com_C.m, varphi_sh_com_C.m, varphi_u_com_C.m,varphi_u_com_C.m*1.5 ])


fig_m_phi_b500c, ax_m_phi_b500c = plt.subplots()


ax_m_phi_b500c.plot(phi_b500c, momente_b500c, color='black')

# Add labels and show the plot
ax_m_phi_b500c.set_ylabel("$M$ [kNm]")
ax_m_phi_b500c.set_xlabel("$\\varphi$ [rad]")

# Save the plot to the specified file
fig_m_phi_b500c.savefig("imgs/jag_m_phi_b500c.svg")
plt.close()

In [98]:

varphi_max_com_C = varphi_max_com_C.to(un.rad)

# put_out(symbolic=False,precision=3, rows = 1)

In [99]:
DF_b500c_res = pd.read_excel('calc/axis_res.xlsx', sheet_name='jag_b500c')

DF_b500c_res['Lpa'] = DF_b500c_res['Lpa']*30
DF_b500c_res['eZ_1 [mm]'] = DF_b500c_res['eZ_1 [mm]']*-1



DF_b500a_res = pd.read_excel('calc/axis_res.xlsx', sheet_name='jag_b500a')

DF_b500a_res['Lpa'] = DF_b500a_res['Lpa']*30
DF_b500a_res['eZ_1 [mm]'] = DF_b500a_res['eZ_1 [mm]']*-1




In [100]:

w_A_array = np.array([0., w_SA1.m, w_SA3.m])
q_A_array = np.array([0.,q_SA1.m, q_SA3.m])


w_C_array = np.array([0.,w_SC1.m, w_SC2.m, w_SC3.m])
q_C_array = np.array([0.,q_SC1.m, q_SC2.m, q_SC3.m])

In [101]:
fig_q_w, ax_q_w = plt.subplots(figsize=(14/2.54,5.4/2.54))


ax_q_w.plot(w_A_array, q_A_array, color='black', label='B500A analytisch')
ax_q_w.plot(DF_b500a_res['eZ_1 [mm]'], DF_b500a_res['Lpa'], color='black', linestyle='dashed', label='B500A FEM')

ax_q_w.plot(w_C_array, q_C_array, color='gray', label='B500C analytisch')
ax_q_w.plot(DF_b500c_res['eZ_1 [mm]'], DF_b500c_res['Lpa'], color='gray', linestyle='dashed', label='B500C FEM')

# Add labels and show the plot
ax_q_w.set_ylabel("$q$ [kN/m]")
ax_q_w.set_xlabel("$w$ [mm]")

plt.legend()
# Save the plot to the specified file
fig_q_w.savefig("imgs/jag_q_w.svg")
plt.close()